In [2]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

# 1 prepare dataset

batch_size = 64

# ToTensor  转换图片为张量
# Normalized an tensor image with mean and standard deviation
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.MNIST(root='./data/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='./data/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# 2 design model

model = nn.Sequential(
    # 卷积1操作
    # input: channel=1, out_channel=10, kernel_size5*5;
    # output: h*w=(28-5+0+1)/1*(28-5+0+1)/1=24*24
    nn.Conv2d(1, 10, kernel_size=5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    # 卷积2操作 接收池化后的conv1 input(h*w)=h*w/2=24/2=12
    # input: channel=10, out_channel=20, kernel_size5*5;
    # output: h*w=(12-5+0+1)/1*(12-5+0+1)/1=8*8
    nn.Conv2d(10, 20, kernel_size=5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.Flatten(),
    # 全连接 接收池化后的conv2 input(h*w)=h*w/2=8/2=4
    # input: 展开后为20*(4*4)=320
    # output: 10 手写字10个分类
    nn.Linear(320, 10))

# 3 交叉熵损失函数 优化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


# optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)

# 4 training cycle forward, backward, update
def train(epoch):
    running_loss = 0.0

    for step, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if step % 300 == 299:
            correct = 0
            total = 0
            with torch.no_grad():
                for l_data in test_loader:
                    images, labels = l_data
                    l_outputs = model(images)
                    _, predicted = torch.max(l_outputs.data, dim=1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            print(
                'Epoch: %d |train loss: %.3f |accuracy: %d %%' % (epoch + 1, running_loss / 300, 100 * correct / total))
            running_loss = 0.0


def jn_cnn():
    for epoch in range(10):
        train(epoch)

    torch.save(model, './jn_cnn')


jn_cnn()

# import torchvision.models as models
# vgg16 = models.vgg16(pretrained=True)
# vgg16.train()


Epoch: 1 |train loss: 1.559 |accuracy: 85 %
Epoch: 1 |train loss: 0.487 |accuracy: 90 %
Epoch: 1 |train loss: 0.329 |accuracy: 91 %
Epoch: 2 |train loss: 0.261 |accuracy: 93 %
Epoch: 2 |train loss: 0.228 |accuracy: 94 %
Epoch: 2 |train loss: 0.203 |accuracy: 95 %
Epoch: 3 |train loss: 0.182 |accuracy: 95 %
Epoch: 3 |train loss: 0.155 |accuracy: 95 %
Epoch: 3 |train loss: 0.146 |accuracy: 96 %
Epoch: 4 |train loss: 0.138 |accuracy: 96 %
Epoch: 4 |train loss: 0.124 |accuracy: 96 %
Epoch: 4 |train loss: 0.114 |accuracy: 96 %
Epoch: 5 |train loss: 0.111 |accuracy: 97 %
Epoch: 5 |train loss: 0.102 |accuracy: 97 %
Epoch: 5 |train loss: 0.104 |accuracy: 97 %
Epoch: 6 |train loss: 0.093 |accuracy: 97 %
Epoch: 6 |train loss: 0.094 |accuracy: 97 %
Epoch: 6 |train loss: 0.086 |accuracy: 97 %
Epoch: 7 |train loss: 0.077 |accuracy: 97 %
Epoch: 7 |train loss: 0.081 |accuracy: 97 %
Epoch: 7 |train loss: 0.086 |accuracy: 97 %
Epoch: 8 |train loss: 0.080 |accuracy: 97 %
Epoch: 8 |train loss: 0.072 |acc

In [6]:
from __future__ import print_function
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch import optim
from torch.utils.data import DataLoader
import datetime


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: ", device)
#
batchSize = 4

##load data
transform = transforms.Compose([transforms.Resize(224), transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.FashionMNIST(root='./data/fashion/', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batchSize, shuffle=True, num_workers=0)

testset = torchvision.datasets.FashionMNIST(root='./data/fashion/', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batchSize, shuffle=False, num_workers=0)

# 导入预训练模型
vgg16 = torchvision.models.vgg16(pretrained=True)
# 打印vgg16结构
print(vgg16)


class JnVgg16(nn.Module):
    def __init__(self):
        super(JnVgg16, self).__init__()

        # 预训练vgg16的特征提取层
        self.features = vgg16.features
        self.features[0] = nn.Conv2d(1, 64, 3, 1, 1)

        self.avgpool = vgg16.avgpool
        # 添加新的全连接层
        self.classifier = nn.Sequential(
            nn.Linear(25088, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),  # 防止过拟合
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 10)
        )

    # 定义前向传播路径
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        # x = x.view(-1, 25088)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


# 输出网络结构


jn_vgg = JnVgg16()
jn_vgg.to(device)
# net = vgg16
print(jn_vgg)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(jn_vgg.parameters(), lr=0.05, momentum=0.9)
optimizer = optim.SGD(jn_vgg.parameters(), lr=0.01)


# train
def train(epoch):
    running_loss = 0.0
    start = datetime.datetime.now()
    total_step = len(trainloader)
    for step, data in enumerate(trainloader, 0):
        inputs, target = data

        optimizer.zero_grad()
        if 'cpu' != device.type:
            inputs = inputs.cuda()
            target = target.cuda()
        outputs = jn_vgg(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if step % 1000 == 999:
            end = datetime.datetime.now()

            interval = end - start

            print('Epoch: %d |step: %d / %d |train loss: %.3f |cost time: %d s' % (epoch + 1, step, total_step, running_loss / 1000, interval.seconds))
            running_loss = 0.0
            start = datetime.datetime.now()


def test(epoch):
    model = jn_vgg
    model.to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for l_data in testloader:
            images, labels = l_data
            if 'cpu' != device.type:
                images = images.cuda()
                labels = labels.cuda()
            l_outputs = model(images)
            _, predicted = torch.max(l_outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Epoch: %d |accuracy: %d %% ' % (epoch + 1, 100 * correct / total))


def jn_vgg16_run():
    for epoch in range(5):
        train(epoch)
        test(epoch)


    # torch.save(jn_vgg, './model/jn_vgg16')


jn_vgg16_run()


device:  cuda:0
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, paddi